# Analysis of the LLM Runs
This will use the the event_study file to analyse the performance across the different strategies. The strategies tested in this file are:

- Base case (consensus estimates from Wall Street analysts on the sellside)
- Llama (zero shot)
- Llama (chain of thought)

In [127]:
import json
import pandas as pd
import numpy as np
import importlib
import event_study as es

from bloomberg.bquant.signal_lab.workflow.utils import get_sandbox_path
from backtest_params import get_analyst_params, get_universe_params

import plotly.express as px

datapack_path = f"{get_sandbox_path()}/tmp/fs/datapack"

In [121]:
importlib.reload(es)

<module 'event_study' from '/project/event_study.py'>

## Base case

This is used as the baseline and is the consensus estimates based on Wall Street sellside analysts. The recommendations are aggregated across a number of analysts. If the rating is 5, this is a strong buy, if the rating is a 1, this is a strong sell. 

In [116]:
# STEP 1: Get the data 
start = '2019-03-31'
end = '2024-12-31'
universe_name = 'INDU Index'
universe, benchmark, trading_calendar = get_universe_params(start, end, universe_name, datapack_path)
analyst_ratings = get_analyst_params(start, end, datapack_path)

In [117]:
analyst_ratings.bind_universe(universe)
ratings_df = analyst_ratings.df().ffill()

100%|██████████| 1/1 [00:06<00:00,  6.26s/it]


In [118]:
# STEP 2: Convert the recommendations into a list of trades
def rating(rating: float) -> int:
    if rating > 3.5:
        return 1
    if rating < 3.0:
        return -1
    else:
        return 0

def trade_direction(trade: int) -> str:
    if trade == 1:
        return "BUY"
    if trade == -1:
        return "SELL"
    else:
        return "HOLD"

def convert_to_trade_view(df: pd.DataFrame) -> pd.DataFrame:
    trade_dfs = []
    # loop through each column
    for security in ratings_df.columns:
        # convert to a numerical buy/ sell/ hold
        df[security] = df[security].apply(rating)
        diff_df = df[[security]]
        diff_df['diff'] = diff_df[security].diff().fillna(1)
        # look for where the rating has changed and filter on this date
        trade_df = diff_df[diff_df['diff'] != 0]
        # change back to BUY/ SELL/ HOLD
        trade_df[security] = trade_df[security].apply(trade_direction)
        # reformat the dataframe so that it is Date | Security | trade direction
        trade_df['Security'] = security
        final_df = trade_df.reset_index().rename(columns={security:'Decision', 'DATE':'Date'})[['Date','Security', 'Decision']].reset_index()
        trade_dfs.append(final_df)
    return pd.concat(trade_dfs)

trade_analyst_recommendations = convert_to_trade_view(ratings_df.copy(deep=True))

In [124]:
# STEP 3: Backtest the strategy using the EventBacktester
backtest = es.EventBacktest(start=start, 
                            end=end, 
                            universe_name=universe_name, 
                            data_pack_path=datapack_path)
results = backtest.run(trade_analyst_recommendations, 'base')

100%|██████████| 117/117 [00:48<00:00,  2.39it/s]


In [125]:
results.plot_analytics()

In [126]:
backtest.get_return_data()

(            Cum. Return (Gross)  Cum. Return (Net)  Cum. T-Cost
 DATE                                                           
 2019-04-01             0.000000           0.000000         -0.0
 2019-04-02             0.000000           0.000000         -0.0
 2019-04-03             0.004898           0.004898         -0.0
 2019-04-04             0.009748           0.009748         -0.0
 2019-04-05             0.012150           0.012150         -0.0
 ...                         ...                ...          ...
 2024-12-24             1.300625           1.300625         -0.0
 2024-12-26             1.273881           1.273881         -0.0
 2024-12-27             1.273848           1.273848         -0.0
 2024-12-30             1.294193           1.294193         -0.0
 2024-12-31             1.286551           1.286551         -0.0
 
 [1448 rows x 3 columns],
             Cum. Return (Gross)  Cum. Return (Net)  Cum. T-Cost
 DATE                                                         

#### Llama inference task

In [3]:
# load the data
with open('Results/results - 2025-02-23 13:59:33.728956.json', 'rb') as file:
    base = json.load(file)
    
base['model']

'meta-llama/Llama-3.2-3B-Instruct'

In [4]:
results = base['results']

In [11]:
results[0]

{'date': '2020-02-06',
 'security': 'MMM UN Equity',
 'response': {'decision': 'BUY',
  'confidence score': 70,
  'reason': "Despite a decline in revenue, the company has shown a consistent increase in gross profit over the past few years, which indicates a potential for growth. Additionally, the company's debt-to-equity ratio is relatively low, indicating a healthy financial position. However, the stock price has been volatile in recent months, which may be a concern.",
  'AdditionalContext': "Here is the analysis of the company's financial statements:I have computed the following financial ratios:* Debt-to-Equity Ratio: 0.34 (2020-02-06)* Interest Coverage Ratio: 13.44 (2020-02-06)* Return on Equity (ROE): 14.15% (2020-02-06)* Return on Assets (ROA): 8.55% (2020-02-06)These ratios suggest that the company is in a relatively healthy financial position, but the volatile stock price and declining revenue are concerns. The decision to buy is based on the potential for growth indicated by

In [86]:
response = base[0]['response']
response.find('HOLD')

1321

In [23]:
def unpack_to_dataframe(data):
    date = []
    sec = []
    decision = []
    confidence = []
    
    for result in data:
        try:
            date.append(result['date'])
            sec.append(result['security'])
            response = result['response']
            try:
                decision.append(response['decision'])
                confidence.append(response['confidence score'])
            except:
                try:
                    #response = result['response']

                    isBuy = False
                    isSell = False
                    isHold = False

                    if response.find('BUY') != -1:
                        isBuy = True
                    if response.find("SELL") != -1:
                        isSell = True
                    if response.find("HOLD") != -1:
                        isHold = True

                    if isBuy and not(isSell or isHold):
                        decision.append('BUY')
                        confidence.append(np.nan)
                        continue
                    if isSell and not(isBuy or isHold):
                        decision.append('SELL')
                        confidence.append(np.nan)
                        continue
                    if isHold and not(isBuy or isSell):
                        decision.append('HOLD')
                        confidence.append(np.nan)
                        continue

                    decision.append('Missing')
                    confidence.append(np.nan)
                except:
                    decision.append('Missing')
                    confidence.append(np.nan)
        except:
            print('Missing date')
        
        
        
    return {'Date': date, 'Security': sec, 'Decision': decision, 'Confidence': confidence}
        
            

In [24]:
s = unpack_to_dataframe(results)

In [31]:
s['Decision'][3]

'BUY'

In [32]:
df = pd.DataFrame(data=s)
df.to_csv('test_analysis.csv')

In [27]:

def explore_data(df):
    print("Number Missing: ", df['Decision'].value_counts()['Missing'])
    print("Number of Buy: ", df['Decision'].value_counts()['BUY'])
    print("Number of Sell: ", df['Decision'].value_counts()['SELL'])
    print("Number of Hold: ", df['Decision'].value_counts()['HOLD'])

In [28]:
explore_data(df)

Number Missing:  7
Number of Buy:  863
Number of Sell:  13
Number of Hold:  13


#### Deepseek inference 14B model

In [72]:
# load the data
with open('Data/base_deepseek_r2.json', 'rb') as file:
    ds = json.load(file)

In [115]:
ds[3]

{'security': 'CVX UN Equity',
 'date': '2020-05-06',
 'response': 'Okay, so I need to figure out whether to buy, sell, or hold this company based on the financial data provided. Let me start by looking at the income statement first. The revenue has been fluctuating a bit. It was 2.9705e+10 last year, but in previous years, it was higher, like 4.0338e+10 two years ago. So, revenue isn\'t showing a clear upward trend.\n\nLooking at the gross profit, it\'s been inconsistent too. Last year it was 3.45e+09, but two years ago it was negative 8.504e+09. Wait, that\'s a big swing. Maybe there was some one-time expense or loss then. Gross profit has been positive in the more recent years, which is good, but it\'s not showing consistent growth. It went from 4.62e+09 to 4.445e+09 to 4.92e+09. Hmm, not a clear trend.\n\nOperating income is also fluctuating. Last year it was 2.609e+09, but the year before that, it was negative 1.0003e+10. That\'s a huge drop. So, the company had a significant loss 

In [112]:
s = unpack_to_dataframe(ds)
df1 = pd.DataFrame(data=s)
df1

,Date,Security,Decision,Confidence
0,2020-05-05,DD UN Equity,SELL,75.0
1,2020-05-06,XOM UN Equity,SELL,75.0
2,2020-05-06,UNH UN Equity,SELL,75.0
3,2020-05-06,CVX UN Equity,HOLD,NaN
4,2020-05-07,PFE UN Equity,SELL,85.0
...,...,...,...,...
891,2025-01-31,XOM UN Equity,SELL,85.0
892,2025-02-04,PFE UN Equity,HOLD,75.0
893,2025-02-04,MRK UN Equity,Missing,NaN
894,2025-02-04,AMGN UW Equity,Missing,NaN


In [113]:
explore_data(df1)

Number Missing:  112
Number of Buy:  87
Number of Sell:  267
Number of Hold:  430
